In [7]:
import os
import sys

sys.path.append("../")  # Adds higher directory to python modules path.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
from core.ODEDataset import ODEDataset

from utils import modifiers
from utils import constants
import pandas as pd


# Clusters

In [2]:
cooking_hrs_cluster = [
    "i_23_hrs_morning",
    "i_24_hrs_afternoon",
    'i_25_hrs_evening',
    'i_26_hrs_h2o'
]

# Constants



# Categories

In [3]:
Main_occupation_original2final = {
    'Unemployed': 'Unemployed',
    'Student': 'Unemployed',
    'Wage Employee, Non-Farm': 'Employee non-farm',
    'Self-Employed Agriculture/Livestock': 'Own-account worker farm',
    'Casual/Day Laborer': 'Worker not classifiable by status',
    'Self-Employed Non-Farm-Business enterprise': 'Own-account worker non-farm',
    'Job Seeker': 'Unemployed',
    'Wage Employee, Farm': 'Employee farm',
    'Too old to work': 'Unemployed',
    'Self-Employed Non-Farm-Independent contractor,technician, professional, etc.': 'Own-account worker non-farm',
    'Retired/pensioner': 'Unemployed',
    'Assistance in family enterprise': 'Contributing family worker',
    'Other': 'Worker not classifiable by status',
    'Disabled': 'Unemployed',
    'Intern/free labor/voluntary work': 'Worker not classifiable by status'
}

Education_level_original2final = {
    'None': 'No schooling',
    'Primary': 'Primary education',
    'Secondary School': 'Lower secondary education',
    'Vocational/Technical/School- after completing primary education': 'Post-secondary non-tertiary education',
    'Vocational/Technical/School- after completing secondary education': 'Post-secondary non-tertiary education',
    'University': "Bachelor's or equivalent level",
    'Post-Grad': "Doctoral or equivalent level",
    'Other': 'Not elsewhere classified'
}


In [12]:

# MTF_HH_Roster
MTF_HH_Roster = ODEDataset("keyna/MTF_HH_Roster")
MTF_HH_Roster.from_csv("../playground/data/ESMAP/kenya/MTF_HH_Roster.csv").group_by("cluuq")

MTF_HH_Roster = MTF_HH_Roster.new_feature("Age_HHH", modifiers.extract_age_of_head('a_5_age', 'a_4_rel_hhh', 'Head'))

MTF_HH_Roster = MTF_HH_Roster.new_feature('Main_occupation',
                                          modifiers.categorize("a_14_main_occ", Main_occupation_original2final))

MTF_HH_Roster = MTF_HH_Roster.new_feature("HH_with_home_business",
                                          modifiers.extract_head_w_home_business('a_19_hh_ent', 'Main_occupation',
                                                                                 'a_5_age', 'Yes'))

MTF_HH_Roster = MTF_HH_Roster.new_feature('Education_level_HHH',
                                          modifiers.categorize('a_9a_sch_lev', Education_level_original2final))

MTF_HH_Roster = MTF_HH_Roster.new_feature('Number_workers', modifiers.extract_working_people('Main_occupation'))
MTF_HH_Roster = MTF_HH_Roster.new_feature('Socio_status_HHH',
                                          modifiers.extract_socio_status_hhh('Kenya', 'Main_occupation', 'a_4_rel_hhh',
                                                                             'Head'))

MTF_HH_Roster = MTF_HH_Roster.new_feature('Number_adults', modifiers.extract_age_groups('a_5_age', 'adults'))

In [17]:
MTF_HH_Roster = MTF_HH_Roster.select(
    ['cluuq', 'Age_HHH', 'Number_workers', 'Socio_status_HHH', 'Number_adults'])

MTF_HH_Roster.to_dataframe().head()

,cluuq,Age_HHH,Number_workers,Socio_status_HHH,Number_adults
0,0,50.0,NaN,Employee non-farm,15.0
1,1001_3,NaN,NaN,NaN,34.0
2,1026_3,NaN,NaN,NaN,59.0
3,1027_2,NaN,NaN,NaN,57.0
4,1029_3,NaN,NaN,NaN,29.0


# MTF_HH_Cooking_Data_Final

In [18]:

MTF_HH_Cooking_Data_Final = ODEDataset("keyna/MTF_HH_Cooking_Data_Final")
MTF_HH_Cooking_Data_Final.from_csv("../playground/data/ESMAP/kenya/MTF_HH_Cooking_Data_Final.csv").group_by("cluuq")

MTF_HH_Cooking_Data_Final = MTF_HH_Cooking_Data_Final.new_feature('Fuel_usage',
                                                                  modifiers.extract_fuel_usage(cooking_hrs_cluster))

MTF_HH_Cooking_Data_Final = MTF_HH_Cooking_Data_Final.new_feature('Clean_fuel',
                                                                  modifiers.is_clean_fuel(constants.CLEAN_FUELS,
                                                                                          'i_18_a_1st_fuel'))


MTF_HH_Cooking_Data_Final.to_dataframe().head()

MTF_HH_Cooking_Data_Final = MTF_HH_Cooking_Data_Final.select(['cluuq','Clean_fuel'])


In [19]:
Kenya = MTF_HH_Roster.merge(MTF_HH_Cooking_Data_Final,'cluuq')
Kenya.to_dataframe().head()

,cluuq,Age_HHH,Number_workers,Socio_status_HHH,Number_adults,Clean_fuel
0,0,50.0,NaN,Employee non-farm,15.0,No
1,1001_3,NaN,NaN,NaN,34.0,No
2,1026_3,NaN,NaN,NaN,59.0,No
3,1027_2,NaN,NaN,NaN,57.0,No
4,1029_3,NaN,NaN,NaN,29.0,No
